# Python + Pandas is love

L'idéal pour une rapport de données, la validation d'une intuition, vite fait, sur un coin de table, en mode oneshot / quick&dirty ? ~~Excel~~ [Pandas](https://pandas.pydata.org/) !

Nous allons suivre la génération d'un rapport à partir d'un _petit_ dump de données, ce qui va nous permettre de faire une visite guidée des fonctionnalités de pandas. Ce ne sera pas exhaustif, mais devrait vous permettre d'y penser la prochaine fois. 


## La base


In [15]:
%matplotlib inline
import pandas as pd
import numpy as np

### Serie

Une série, est une séquence de valeurs, de type homogène (toutes du même type : String, entiers, flotants, dates ... ), __ avec un index __

In [7]:
s = pd.Series([5,4,3,2,1,2,3,4,5])
s_datetimes = pd.date_range(start="2017-01-01", periods=86400, freq="2H").to_series()

display(s.head())
display(s_datetimes.head())

0    5
1    4
2    3
3    2
4    1
dtype: int64

2017-01-01 00:00:00   2017-01-01 00:00:00
2017-01-01 02:00:00   2017-01-01 02:00:00
2017-01-01 04:00:00   2017-01-01 04:00:00
2017-01-01 06:00:00   2017-01-01 06:00:00
2017-01-01 08:00:00   2017-01-01 08:00:00
Freq: 2H, dtype: datetime64[ns]

## Dataframe

Un Dataframe, c'est un tableau : 
- un ensemble de Series, nommées ( les colonnes)
- qui partagent un index


In [22]:
data = pd.DataFrame({"time": s_datetimes[:120].values,  # get rid of the index
                     "value": np.random.random(120),
                     "category": pd.Categorical(list("abcdef" * 20))})

print(data.info(()))
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 3 columns):
category    120 non-null category
time        120 non-null datetime64[ns]
value       120 non-null float64
dtypes: category(1), datetime64[ns](1), float64(1)
memory usage: 2.3 KB
None


,category,time,value
0,a,2017-01-01 00:00:00,0.503767
1,b,2017-01-01 02:00:00,0.171931
2,c,2017-01-01 04:00:00,0.223380
3,d,2017-01-01 06:00:00,0.961785
4,e,2017-01-01 08:00:00,0.723667
